In [1]:
# import os
# os.chdir("..")

In [ ]:
import os
import uuid
import cv2
import json
from ultralytics import YOLO
from datetime import datetime, timedelta
import numpy as np

def analyze_video(
    video_path: str,
    video_id: str,
    video_start_timestamp: str,
    region_definitions: list,
    model_name: str = "yolo11n.pt",
    tracker: str = "botsort.yaml",
    classes: list = [0],
    max_frames: int = -1,
    save: bool = True
):
    job_id = str(uuid.uuid4())

    cap = cv2.VideoCapture(video_path)
    assert cap.isOpened(), "Error reading video file"
    model = YOLO(model_name)
    class_names = model.names

    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps == 0:
        fps = 30  # fallback default

    frame_number = 0
    results_json = []
    video_filename = os.path.basename(video_path)

    # konversi video_start_timestamp ke datetime object
    video_start_dt = datetime.fromisoformat(video_start_timestamp.replace("Z", "+00:00"))

    # timestamp job deteksi
    job_timestamp = datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or (max_frames > 0 and frame_number >= max_frames):
            break

        # timestamp frame sekarang
        current_timestamp = video_start_dt + timedelta(seconds=frame_number / fps)
        timestamp_iso = current_timestamp.isoformat() + "Z"

        result = model.track(
            frame,
            persist=True,
            conf=0.25,
            verbose=False,
            tracker=tracker,
            classes=classes
        )[0]

        for region in region_definitions:
            frame_objects = []
            count = 0  # inisialisasi counter

            boxes = result.boxes
            for i in range(len(boxes)):
                cls_id = int(boxes.cls[i].item())
                cls_name = class_names[cls_id]
                if cls_name not in [class_names[i] for i in classes]:
                    continue

                bbox = boxes.xyxy[i].tolist()
                track_id = int(boxes.id[i].item()) if boxes.id is not None else None
                conf = float(boxes.conf[i].item())
                center_x = (bbox[0] + bbox[2]) / 2
                center_y = (bbox[1] + bbox[3]) / 2

                inside = cv2.pointPolygonTest(
                    np.array(region['polygon'], dtype=np.int32),
                    (center_x, center_y),
                    False
                ) >= 0

                if inside:
                    count += 1

                frame_objects.append({
                    "id": track_id,
                    "bbox": [int(v) for v in bbox],
                    "class": cls_name,
                    "confidence": conf,
                    "inside_region": inside
                })

            results_json.append({
                "video_id": video_id,
                "video_filename": video_filename,
                "frame_number": frame_number,
                "timestamp": timestamp_iso,
                "region_id": region['id'],
                "region_name": region['name'],
                "region_description": region['description'],
                "region_polygon": region['polygon'],
                "objects": frame_objects,
                "count": count
            })

        frame_number += 1

    cap.release()

    if save:
        output_filename = f"detection_results_{job_id}_{job_timestamp}.json"
        output_path = os.path.join(os.path.dirname(video_path), output_filename)
        with open(output_path, "w") as f:
            json.dump(results_json, f, indent=2)
            
        print(output_path)

    return results_json


In [3]:
region_definitions = [
    {
        "id": 1,
        "name": "Pintu Masuk A",
        "description": "Pintu utama gedung A",
        "polygon": [(1000, 500), (1300, 500), (1300, 1000), (1000, 1000)]
    },
    {
        "id": 2,
        "name": "Pintu Keluar B",
        "description": "Area keluar belakang gedung B",
        "polygon": [(500, 500), (800, 500), (800, 1000), (500, 1000)]
    },
    # {
    #     "id": 3,
    #     "name": "Pintu Keluar C",
    #     "description": "Area keluar belakang gedung B",
    #     "polygon": [(500, 500), (1300, 500), (1300, 1000), (500, 1000)]
    # }
]


In [4]:
video_id = "32b583ef-ae36-42a9-9f8e-9ad7a5501970"
video_path = r"output\32b583ef-ae36-42a9-9f8e-9ad7a5501970\kepatihan_20250804_064451.mp4"
video_start = "2025-08-04T06:45:10.718263"

json_results = analyze_video(video_path, video_id, video_start, region_definitions, max_frames=-1, save=True)


C:\Users\Irfan Muhammad Ghani\AppData\Local\Temp\ipykernel_15956\4115260328.py:39: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  job_timestamp = datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")


output\32b583ef-ae36-42a9-9f8e-9ad7a5501970\detection_results_d83229da-7d54-4829-a069-de404a412494_20250803T235016Z.json
